In [5]:
import psycopg2

# Database connection parameters
DATABASE_TYPE = 'postgresql'
DB_USER = 'postgres'  # Your PostgreSQL username
DB_PASSWORD = 'BestPasswordEver'  # Your PostgreSQL password
DB_HOST = '34.133.41.198'  # External IP of your VM or database host
DB_PORT = '5432'  # Default PostgreSQL port
DB_NAME = 'postgres'  # Database name

# Connect to your PostgreSQL database
try:
    connection = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    cursor = connection.cursor()
    print("Connected to the database")

    # Delete from location table where city is not Islamabad or Rawalpindi
    cursor.execute("""
        DELETE FROM location
        WHERE city NOT IN ('Islamabad', 'Rawalpindi');
    """)
    print("Deleted non-Islamabad and non-Rawalpindi locations")

    # Delete from agency table if agencies are linked to locations
    cursor.execute("""
        DELETE FROM agency
        WHERE agency_id NOT IN (
            SELECT DISTINCT agency_id FROM property WHERE location_id IN (
                SELECT location_id FROM location WHERE city IN ('Islamabad', 'Rawalpindi')
            )
        );
    """)
    print("Deleted agencies not linked to Islamabad or Rawalpindi properties")

    # Delete from agent table if agents are linked to agencies
    cursor.execute("""
        DELETE FROM agent
        WHERE agent_id NOT IN (
            SELECT DISTINCT agent_id FROM agency WHERE agency_id IN (
                SELECT agency_id FROM property WHERE location_id IN (
                    SELECT location_id FROM location WHERE city IN ('Islamabad', 'Rawalpindi')
                )
            )
        );
    """)
    print("Deleted agents not linked to Islamabad or Rawalpindi agencies")

    # Delete from property table where location is not in Islamabad or Rawalpindi
    cursor.execute("""
        DELETE FROM property
        WHERE location_id NOT IN (
            SELECT location_id FROM location WHERE city IN ('Islamabad', 'Rawalpindi')
        );
    """)
    print("Deleted properties not located in Islamabad or Rawalpindi")

    # Commit the changes
    connection.commit()
    print("Changes committed successfully")

except Exception as e:
    print("An error occurred:", e)
finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()
    print("Database connection closed")

Connected to the database
Deleted non-Islamabad and non-Rawalpindi locations
Deleted agencies not linked to Islamabad or Rawalpindi properties
Deleted agents not linked to Islamabad or Rawalpindi agencies
Deleted properties not located in Islamabad or Rawalpindi
Changes committed successfully
Database connection closed
